In [ ]:
import pandas as pd
import os
from cdcqr.data.dataloader import data_loader
from cdcqr.common.stats_utils import winsorize_df, remove_outlier
import seaborn as sns
import numpy as np
from cdcqr.data.deribit.data_utils import DeribitUtils
import time
from cdcqr.common.utils import print_time_from_t0
import warnings
warnings.filterwarnings("ignore")

#### load quote data for options

In [ ]:

date = "2021-10-01"
data_type = "quotes" #"quotes"

In [ ]:
df1= data_loader('deribit',"2021-10-01","quotes",'BTC-PERPETUAL')

In [ ]:
df1

#### load trade data for options

In [ ]:
df_opt_quote = data_loader('deribit',"2021-10-01","quotes",'options').pipe(DeribitUtils.parse_optSymbol_col).pipe(DeribitUtils.parse_time_col)
df_opt_quote.head(2)

In [ ]:
df_opt_trade = data_loader('deribit',"2021-10-01","trades",'options').pipe(DeribitUtils.parse_optSymbol_col).pipe(DeribitUtils.parse_time_col)
df_opt_trade.head(2)

In [ ]:
df_opt_trade['dvol'] = df_opt_trade['price']*df_opt_trade['amount']

In [ ]:
df_opt_trade['symbol'].unique()

### check liquidity - number of trades

In [ ]:
trade_info = df_opt_trade.groupby('symbol')['dvol'].agg(['sum','count']).sort_values(['count','sum'], ascending=[False,False]).reset_index()
trade_info.head()

In [ ]:
df_opt_trade.query('symbol=="ETH-24JUN22-4000-C"')

In [ ]:
trade_info['type'] = trade_info['symbol'].str.contains('-C')
trade_info.head()

In [ ]:
sns.scatterplot(x="count", y="sum", data=trade_info, hue="type")

In [ ]:
trade_info_rmoutlier = remove_outlier(trade_info, cols2limits={'count':[0.1,0.9],'sum':[0.1,0.9]})

In [ ]:
sns.scatterplot(x="count", y="sum", data=trade_info_rmoutlier, hue="type")

### identify trade/remove liquidity

In [ ]:
ATM_btc_c = 'BTC-8OCT21-48000-C'
df_opt_trade_i = df_opt_trade.query('symbol==@ATM_btc_c')
df_opt_trade_i.head()
display(df_opt_trade_i.shape)

In [ ]:
df_opt_trade_i['flow_sgn'] = (df_opt_trade_i['side']=='buy').astype(int)
df_opt_trade_i['flow_sgn'] = (df_opt_trade_i['flow_sgn']-0.5)*2
df_opt_trade_i['flow'] = df_opt_trade_i['flow_sgn']*df_opt_trade_i['amount']
df_opt_trade_i_no_dup = df_opt_trade_i.groupby('timestamp_dt')['flow'].sum().reset_index()

In [ ]:
df_opt_trade_i_no_dup.shape

In [ ]:
df_opt_quote_i = df_opt_quote.query('symbol==@ATM_btc_c')
df_opt_quote_i_no_dup = df_opt_quote_i[~df_opt_quote_i['timestamp_dt'].duplicated(keep='last')]
df_opt_quote_i_no_dup.head()

In [ ]:
df_opt_quote_i_no_dup.shape

In [ ]:
df_opt_trade_i_no_dup = df_opt_trade_i_no_dup.rename(columns={'timestamp_dt':'trade_time'})

In [ ]:
B = df_opt_quote_i_no_dup['timestamp_dt'].values
A = df_opt_trade_i_no_dup['trade_time'].values
res = B[np.searchsorted(B, A)]
df_opt_trade_i_no_dup['timestamp_dt'] = res

In [ ]:
df_opt_trade_i_no_dup2 = df_opt_trade_i_no_dup.groupby('timestamp_dt')['flow'].sum().reset_index()

df_opt_trade_i_no_dup2['has_trade'] = True

In [ ]:
df_opt_trade_i_no_dup2.head(2)

In [ ]:
df_opt_quote_i_no_dup.head(2)

In [ ]:
df_combined = pd.merge(left=df_opt_quote_i_no_dup, right=df_opt_trade_i_no_dup2, on=['timestamp_dt'], how='left')

In [ ]:
df_combined['flow'] = df_combined['flow'].fillna(0)

df_combined['trade_neighbour'] = df_combined['has_trade']

df_combined['trade_neighbour'] =df_combined['has_trade'].ffill(limit=1).bfill(limit=1).fillna(False)

In [ ]:
df_combined[df_combined['trade_neighbour']].head(2)

In [ ]:
df_combined['mid_price'] = (df_combined['ask_price'] + df_combined['bid_price'])/2
df_combined['wgt_mid_price'] = (df_combined['ask_price']*df_combined['ask_amount'] + df_combined['bid_price']*df_combined['bid_amount'])/(df_combined['ask_amount']+df_combined['bid_amount'])

df_combined['midp_chg'] = df_combined['mid_price'].pct_change()
df_combined['wgtmidp_chg'] = df_combined['wgt_mid_price'].pct_change()

In [ ]:
display('num trades: {}'.format(df_combined['has_trade'].sum()))
display('midp_chg: {}'.format((df_combined['midp_chg']!=0).sum()))
display('wgtmidp_chg: {}'.format((df_combined['wgtmidp_chg']!=0).sum()))

In [ ]:
def get_order_book_dynamics(contract='BTC-8OCT21-48000-C', date ="2021-10-01" , symbol='options', exchange='deribit'):
    display('BTC-8OCT21-48000-C', "2021-10-01", 'deribit')
    df_opt_quote = data_loader(exchange, date,"quotes",symbol).pipe(DeribitUtils.parse_optSymbol_col).pipe(DeribitUtils.parse_time_col)
    df_opt_trade = data_loader(exchange, date,"trades",symbol).pipe(DeribitUtils.parse_optSymbol_col).pipe(DeribitUtils.parse_time_col)
    ATM_btc_c = contract
    
    # processing trade info
    df_opt_trade_i = df_opt_trade.query('symbol==@ATM_btc_c')
    df_opt_trade_i['flow_sgn'] = (df_opt_trade_i['side']=='buy').astype(int)
    df_opt_trade_i['flow_sgn'] = (df_opt_trade_i['flow_sgn']-0.5)*2
    df_opt_trade_i['flow'] = df_opt_trade_i['flow_sgn']*df_opt_trade_i['amount']
    df_opt_trade_i_no_dup = df_opt_trade_i.groupby('timestamp_dt')['flow'].sum().reset_index()
    df_opt_trade_i_no_dup = df_opt_trade_i_no_dup.rename(columns={'timestamp_dt':'trade_time'})
    
    
    # processing quote info
    df_opt_quote_i = df_opt_quote.query('symbol==@ATM_btc_c')
    df_opt_quote_i_no_dup = df_opt_quote_i[~df_opt_quote_i['timestamp_dt'].duplicated(keep='last')]
    
    # align timestamps
    B = df_opt_quote_i_no_dup['timestamp_dt'].values
    A = df_opt_trade_i_no_dup['trade_time'].values
    res = B[np.searchsorted(B, A)]
    df_opt_trade_i_no_dup['timestamp_dt'] = res
    
    df_opt_trade_i_no_dup2 = df_opt_trade_i_no_dup.groupby('timestamp_dt')['flow'].sum().reset_index()
    df_opt_trade_i_no_dup2['has_trade'] = True
    
    # combine
    df_combined = pd.merge(left=df_opt_quote_i_no_dup, right=df_opt_trade_i_no_dup2, on=['timestamp_dt'], how='left')
    
    # adding features
    df_combined['flow'] = df_combined['flow'].fillna(0)
    df_combined['trade_neighbour'] = df_combined['has_trade']
    df_combined['trade_neighbour'] =df_combined['has_trade'].ffill(limit=1).bfill(limit=1).fillna(False)
    
    df_combined['mid_price'] = (df_combined['ask_price'] + df_combined['bid_price'])/2
    df_combined['wgt_mid_price'] = (df_combined['ask_price']*df_combined['ask_amount'] + df_combined['bid_price']*df_combined['bid_amount'])/(df_combined['ask_amount']+df_combined['bid_amount'])
    df_combined['midp_chg'] = df_combined['mid_price'].pct_change()
    df_combined['wgtmidp_chg'] = df_combined['wgt_mid_price'].pct_change()
    
    num_trades = df_combined['has_trade'].sum()
    num_midp_chg = (df_combined['midp_chg']!=0).sum()
    num_wgtmidp_chg = (df_combined['wgtmidp_chg']!=0).sum()
    return {'num_trades':num_trades,'num_midp_chg':num_midp_chg,'num_wgtmidp_chg':num_wgtmidp_chg}, df_combined

In [ ]:
ret_dict, ret_df = get_order_book_dynamics()

In [ ]:
ret_dict